# 07. 統合演習: RAGとLoRAを組み合わせたGradio UIの作成

このノートブックは、本演習の総まとめです。
これまで学んできた以下の技術をすべて組み合わせ、一つのインタラクティブなアプリケーションを構築します。

- **LLM**: 中核となる言語モデル。
- **RAG**: 外部知識を与え、事実に基づいた回答を生成させる技術。
- **LoRA**: 特定のタスク（JSON出力）に特化させるためのファインチューニング技術。
- **Gradio**: これらの機能を簡単に切り替えられるWeb UI。

## この演習のゴール

**RAGとLoRAのON/OFFを自由に切り替えられるUIを作成し、それぞれの技術がLLMの応答にどのような影響を与えるかをインタラクティブに体験する。**

これにより、「LLMは単なる部品であり、望む結果を得るためには適切なシステム設計が不可欠である」ことを理解します。

## 事前準備

Google Colabで実行する場合、メニューの「ランタイム」→「ランタイムのタイプを変更」で、ハードウェアアクセラレータが「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
import json
from google.colab import drive

if not os.path.isdir('/content/drive'): drive.mount('/content/drive')
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/akio-kobayashi/llm_lab.git
os.chdir(repo_path)

# 必要なライブラリのインストール
!pip install -q -U transformers==4.41.2 accelerate==0.30.1 bitsandbytes==0.43.1 sentence-transformers==2.7.0 faiss-gpu==1.7.2 peft==0.10.0 trl==0.8.6 datasets==2.19.0 gradio==4.31.5
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

try:
    from src.common import load_llm, generate_text
    from src.rag import FaissRAGPipeline
    from src.ui import create_gradio_ui
    from peft import PeftModel
    print('共通モジュールのインポートが完了しました。')
except ImportError as e:
    print(f'共通モジュールのインポートに失敗しました: {e}')

## 1. RAGパイプラインの準備

`04_rag_faiss_exercise` と同様に、Faissインデックスを読み込み、RAGパイプラインを準備します。

In [ ]:
# 編集禁止セル
rag_pipeline = None
try:
    # Google Drive上のパスを指定
    DRIVE_DIR = '/content/drive/MyDrive/llm_lab_outputs'
    INDEX_PATH = os.path.join(DRIVE_DIR, 'faiss_index/anime_docs.index')
    META_PATH = os.path.join(DRIVE_DIR, 'faiss_index/anime_docs_meta.json')
    
    if os.path.exists(INDEX_PATH):
        rag_pipeline = FaissRAGPipeline()
        rag_pipeline.load_index(INDEX_PATH, META_PATH)
        print("RAGパイプラインの準備が完了しました。")
    else:
        print("Faissインデックスが見つかりません。04_rag_faiss_exercise.ipynb を先に実行してください。")
except Exception as e:
    print(f"RAGパイプラインの準備中にエラーが発生しました: {e}")

## 2. ベースモデルとLoRAモデルの準備

ベースとなるLLMと、`06_lora_qlora_exercise` で学習したLoRAアダプタを適用したモデルの両方を準備します。

In [ ]:
# 編集禁止セル
base_model, tokenizer, lora_model = None, None, None
try:
    # ベースモデルのロード
    base_model, tokenizer = load_llm(use_4bit=True)
    
    # Google Drive上のLoRAアダプタのパス
    DRIVE_DIR = '/content/drive/MyDrive/llm_lab_outputs'
    adapter_path = os.path.join(DRIVE_DIR, 'my_lora_adapter/final_adapter')
    
    if os.path.exists(adapter_path):
        # ベースモデルにLoRAアダプタをロード
        lora_model = PeftModel.from_pretrained(base_model, adapter_path)
        lora_model.eval()
        print("LoRAモデルの準備が完了しました。")
    else:
        print(f"LoRAアダプタが見つかりません: {adapter_path}")
        print("LoRAなしモードのみ利用可能です。")

except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')

## 3. Gradio UIのための生成関数を定義

UIからのリクエストに応じて、4つの異なるモードでテキストを生成する関数を定義します。

1.  **Plain**: 素のベースモデル
2.  **RAG**: ベースモデル + RAG
3.  **LoRA**: LoRAモデル
4.  **RAG + LoRA**: LoRAモデル + RAG

In [ ]:
# 編集禁止セル

def generate_plain(query):
    prompt = f"### 指示:\n{query}\n\n### 応答:\n"
    generated_text = generate_text(base_model, tokenizer, prompt)
    return generated_text.split("### 応答:")[-1].strip()

def generate_rag(query):
    context_docs = rag_pipeline.search(query, top_k=3)
    prompt = rag_pipeline.create_prompt_with_context(query, context_docs)
    generated_text = generate_text(base_model, tokenizer, prompt)
    
    answer = generated_text.split("回答:")[-1].strip()
    context_str = "\n\n---\n\n".join([json.dumps(doc, ensure_ascii=False, indent=2) for doc in context_docs])
    return answer, context_str

def generate_lora(query):
    if not lora_model:
        return "(LoRAモデルがロードされていません)", ""
    prompt = f"### 指示:\n{query}\n\n### 応答:\n"
    generated_text = generate_text(lora_model, tokenizer, prompt)
    return generated_text.split("### 応答:")[-1].strip()

def generate_rag_lora(query):
    if not lora_model:
        return "(LoRAモデルがロードされていません)", ""
    context_docs = rag_pipeline.search(query, top_k=3)
    prompt = rag_pipeline.create_prompt_with_context(query, context_docs)
    generated_text = generate_text(lora_model, tokenizer, prompt)
    
    answer = generated_text.split("回答:")[-1].strip()
    context_str = "\n\n---\n\n".join([json.dumps(doc, ensure_ascii=False, indent=2) for doc in context_docs])
    return answer, context_str

print("4つのモードの生成関数を定義しました。")

## 4. Gradio UIの起動

準備した関数を使って、`src/ui.py` の `create_gradio_ui` を呼び出し、UIを起動します。
セルを実行すると、下にUIが表示されます。`Running on public URL...` のリンクをクリックすると、新しいタブでUIを開くこともできます。

In [ ]:
# 編集禁止セル
if all([base_model, tokenizer, rag_pipeline]):
    demo = create_gradio_ui(
        generate_func_plain=generate_plain,
        generate_func_rag=generate_rag,
        generate_func_lora=generate_lora,
        generate_func_rag_lora=generate_rag_lora
    )
    
    # share=Trueで外部URLを生成
    demo.launch(share=True, debug=True)
else:
    print("必要なコンポーネントがロードされていないため、UIを起動できません。")

## まとめと考察

この演習を通じて、LLMを単体で使うだけでなく、RAGやLoRAといった技術と組み合わせることで、その能力を大きく拡張できることを学びました。

- **Plain**: 基本的な対話はできるが、知識が古かったり、嘘をついたりする。
- **RAG**: 知識を外部から与えることで、事実に基づいた正確な回答が可能になる。
- **LoRA**: 特定のタスク（JSON出力など）に特化させ、応答の「スタイル」を制御できる。
- **RAG + LoRA**: 事実に基づき、かつ、望ましいスタイルで応答するという、両方の長所を活かしたシステム。

現代のLLMアプリケーション開発では、このように複数の技術を適切に組み合わせ、一つのシステムとして設計することが非常に重要です。

これで、本演習のすべての内容は終了です。お疲れ様でした！